In [1]:
import matplotlib.pyplot as plt
import json
import os
import sys
import re
 
# Path for spark source folder
os.environ['SPARK_HOME'] = "/usr/local/spark"

# Append pyspark to Python Path
sys.path.append("/usr/local/spark/python")

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
# Load in the testing code and check to see if your answer is correct
# If incorrect it will report back '1 test failed' for each failed test
# Make sure to rerun any cell you change before trying the test again
from test_helper import Test
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# Append afinn to Python Path and import afinn.  Used for pulling data from percentiles.
sys.path.append("/usr/local/lib/python2.7/dist-packages/afinn")
from afinn import Afinn

# Stuff for logistic regression
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.linalg import SparseVector

In [4]:
sys.path.append("/usr/local/lib/python2.7/dist-packages")
from tdigest import TDigest
from numpy.random import random
from operator import add

# 0.0 Read data

Read in data.  For now, on local filesystem.  

<B>ADD CODE TO READ FROM S3.  

df = sqlContext.read.json("/home/ubuntu/RC_2007-10")

In [ ]:
df = sqlContext.read.json("s3n://reddit-comments/")

# 1.0 Find minimum comment timestamp for each post

For each post link_id, find minimum created_utc timestamp [can't trust data is ordered by time] and store in key-value pair (pair RDD) {link_id: min_created_utc},  (Plan B:  set up API to get timestamp for all posts in Reddit)

<B>I'M KEEPING TOO MUCH DATA.  AT THIS STEP, AND SUBSEQUENT STEPS, THROW OUT DATA I'M NOT USING.

In [13]:
rRDD = df.map(lambda r: (r.id, (r.body, int(r.created_utc), r.link_id, r.parent_id, int(r.score), r.subreddit, r.subreddit_id)))

rRDD.take(1)

Sort to find minimum timestamp created_utc for each post link_id

rRDD.count()

In [16]:
minTimeRDD = (rRDD.map(lambda (k, v): (v[2],v[1])) # maps to (link_id, created_utc)
                  .reduceByKey(lambda a, b:  a if a < b else b)
              )

minTimeRDD.take(1)

minTimeRDD.count()

# 2.0 Filter to include only extreme up and down votes (top 3% of subreddit)

Filter to retain only records that are top or bottom 3% in comment score (upvotes-downvotes) of their subreddit.  Reduces dataset for all subsequent processing.

Find 3% and 97% percentiles for each subreddit.  Use T-digest data structure for highly accurate approximate percentiles: 
http://dataorigami.net/blogs/napkin-folding/19055451-percentile-and-quantile-estimation-of-big-data-the-t-digest

In [19]:
def digest1(value):
    digest = TDigest()
    digest.update(value)
    return digest

Need to get at scores for each subreddit, THEN map using digest.


<b>REFACTOR AND MAKE THIS RUN FASTER.  I'M CALLING DIGEST1 TOO MANY TIMES.

ALSO, NEED TO SPLIT INTO TRAINING, VAL AND TEST BEFORE CREATING SUBREDDIT DIGEST ON TRAINING DATA ONLY.  OTHERWISE I'M CHEATING.
</b>

In [20]:
subredditDigestRDD = (rRDD.map(lambda (k, v):  (v[5], v[4]))
                        .map(lambda (k, v): (k, digest1(v)))  
                        .reduceByKey(lambda a, b:  a + b)
                   )

type(subredditDigestRDD)

Creating digest dictionary takes a few minutes.  

In [23]:
subredditDigest = subredditDigestRDD.collectAsMap()

type(subredditDigest['politics'])

subredditDigest['politics'].percentile(3)

subredditDigest['politics'].percentile(97)

Alyssa's values for politics were (-6.0, 24.0) rounded.  

Set limits for what data to include by percentile.  Use only data less than lowPercentile or greater than highPercentile.

In [27]:
lowPercentile, highPercentile = 3, 97

In [28]:
srDigestR = {key : (round(subredditDigest[key].percentile(lowPercentile)), 
                    round(subredditDigest[key].percentile(highPercentile))) 
             for key in subredditDigest.keys()}

srDigestR['politics']

len(srDigestR)

print srDigestR

Filter records to retain only top and bottom 3% of comment scores.

<b>NOTE:  NEED TO BROADCAST srDigestR WHEN I MOVE TO CLUSTER.

In [32]:
rRDDExtreme = rRDD.filter(lambda (k,v): v[4] < srDigestR[v[5]][0] or v[4] > srDigestR[v[5]][1])

rRDDExtreme.count()

# 3.0  Calculate timeSince

Calculate time since post was created based on created_utc and min_created_utc from pair RDD.  In Alyssa's IPython notebook this is called timeSince.  In her R code it's called recency.  

For this I need a left outer join.  For each element (k, v) in self, the resulting RDD will either contain all pairs (k, (v, w)) for w in other, or the pair (k, (v, None)) if no elements in other have key k.  I need to know if there are records in my comment dataset for which there is no "minimum time", as this would indicate a processing error.

Map RDD to get post link_id as key, then join with minTimeRDD.

<b>CAN I USE DATAFRAMES FOR ALL THIS? GETTING TIRED OF REMEMBERING WHAT v[4] IS.  </b>

The only data I need for regression are:  C(subreddit) + timeSince + commentLength + posNegDiff.  Need to keep only comment id (key), score, subreddit, timeSince, comment text from this step.

Format of output RDD is (id,(body,timeSince,score,subreddit))

In [34]:
rRDDXts = (rRDDExtreme.map(lambda (k,v):  (v[2],(k,v[0],v[1],v[2],v[3],v[4],v[5],v[6])))  # pull link_id as key
                      .leftOuterJoin(minTimeRDD) # join on link_id (post)
                      .map(lambda (link_id,(x,min_utc)):  (x[0], (x[1],x[2]-min_utc,x[5],x[6])))
                      .cache()
          )

rRDDXts.take(10)

# 4.0  Calculate commentLength

Clean comment body and calculate commentLength.

R gsub:
gsub(pattern, replacement, x, ignore.case = FALSE, perl = FALSE, fixed = FALSE, useBytes = FALSE)

Python re:
re.sub(pattern, repl, string, count=0, flags=0).  Return the string obtained by replacing the leftmost non-overlapping occurrences of pattern in string by the replacement repl.

<B>NOTE:  ALYSSA REMOVED QUOTED COMMENTS.  I REMOVING THEM ALSO BUT IN MY FIRST EXAMPLE I FOUND A "MADE UP" QUOTE THAT ISN'T REALLY QUOTING SOMEONE ELSE'S POST.  

In [36]:
def cleanup(body):

	# Recode HTML codes
	body = re.sub("&gt;", ">", body)
	body = re.sub("&lt;", "<", body)
	body = re.sub("&amp;", "&", body)
	body = re.sub("&nbsp;", " ", body)

	# Remove deleted
	body = re.sub("^[deleted]$", "", body)

	# Remove URL
	body = re.sub("http[[:alnum:][:punct:]]*", " ", body) # url

	# Remove /r/subreddit, /u/user
	body = re.sub("/r/[[:alnum:]]+|/u/[[:alnum:]]+", " ", body)

	# Remove quoted comments
	body = re.sub("(>.*?\\n\\n)+", " ", body)

	# Remove control characters (\n, \b)
	body = re.sub("[[:cntrl:]]", " ", body)

	# Remove single quotation marks (contractions)
	body = re.sub("'", "", body)

	# Remove punctuation
	body = re.sub("[[:punct:]]", " ", body)

	# Replace multiple spaces with single space
	body = re.sub("\\s+", " ", body) # Multiple spaces
	# body = re.sub("^\\s+", "", body) # Space at the start of the string
	# body = re.sub("+\\s$", "", body) # Space at the end of the string
	body = body.strip()

	# Lower case
	body = body.lower()

	# Return comment length (number of words) and body (cleaned up text)
	return body

clbody = cleanup(u"Basically, the hospital's position amounts to:\n\n&gt; If she can't hold her roofies she deserves to be a**f****d and denied medical care and collection of evidence!\n\nNot the *most* progressive attitude...")

print len(clbody.split())

print clbody

Current format of RDD:  (id,(body,timeSince,score,subreddit))
Format of rRDDXtscl:  (id,(commentLength,body,timeSince,score,subreddit))

In [40]:
rRDDXtscl = (rRDDXts.map(lambda (id,(body,timeSince,score,subreddit)): (id,(cleanup(body),timeSince,score,subreddit)))
                    .map(lambda (id,(body,timeSince,score,subreddit)): (id,(len(body.split()),body,timeSince,score,subreddit)))
             )

# 5.0 (Filter out exclusions if necessary; skip for now)

Filter out exclusions.  Further reduces dataset.

# 6.0 Run sentiment analysis and calculate posNegDiff

Use AFINN model to do sentiment analysis.

Finn Årup Nielsen, "A new ANEW: evaluation of a word list for sentiment analysis in microblogs" , Proceedings of the ESWC2011 Workshop on 'Making Sense of Microposts': Big things come in small packages 718 in CEUR Workshop Proceedings: 93-98. 2011 May. Matthew Rowe, Milan Stankovic, Aba-Sah Dadzie, Mariann Hardey (editors)

<B>I'M WONDERING IF CREATING AN AFINN OBJECT EACH TIME IS TAKING TOO LONG.  MIGHT WANT TO REFACTOR THIS TO A PYTHON FUNCTION WITH A DICT LOOKUP AND USE A BROADCAST VARIABLE.  

In [41]:
def sentiment(body):
    afinn = Afinn()
    return afinn.score(body)

sentiment("This is utterly excellent!")

In [43]:
rRDDtscls = (rRDDXtscl.map(lambda (id,(commentLength,body,timeSince,score,subreddit)):  
                        (id,(commentLength,sentiment(body),timeSince,score,subreddit)))
                      .cache()
             )

rRDDtscls.take(5)

rRDDtscls.count()

# 7.0 Set up logistic regression inputs with OHE features for categorical variable subredddit

Calculate label from score using srDigestR and create rawData RDD in proper format:  (label, non-categorical variables, categorical variable)

In [46]:
def label(score, subreddit, percentMap):
    if score <= percentMap[subreddit][0]: return 0
    else: return 1

Format of rRDDtscls:  (id,(commentLength,posNegDiff,timeSince,score,subreddit)))

Format of rawData is a tuple:  (label, (0,commentLength), (1,posNegDiff), (2,timeSince), subreddit))

def tupToString(tup):
    return ','.join([str(item) for item in tup])

In [47]:
rawData = (rRDDtscls.map(lambda (id,(commentLength,posNegDiff,timeSince,score,subreddit)):  
                    (label(score,subreddit,srDigestR), (0,commentLength), (1,posNegDiff), (2,timeSince), subreddit))
                    .cache()
          )

type(rawData)

rawData.take(5)

rawData.count()

Split data into training, validation and test. 

In [51]:
weights = [.8, .1, .1]
seed = 42
# Use randomSplit with weights and seed
rawTrainData, rawValData, rawTestData = rawData.randomSplit(weights, seed)

# Cache the data
rawTrainData.cache()
rawValData.cache()
rawTestData.cache()

nAll = rawData.count()
nTrain = rawTrainData.count()
nVal = rawValData.count()
nTest = rawTestData.count()
print nTrain, nVal, nTest, nTrain + nVal + nTest, nAll

# print rawData.take(1)

7035 882 843 8760 8760


Create one hot encoding dictionary.

I DON'T NEED ML LAB 4 createOneHotDict BECAUSE I ALREADY HAVE A LIST OF SUBREDDITS IN srDigestR.  Just need to pull keys out of this dict to create my OHEdict. NOTE that my OHEdict is already shifted by 3 to avoid collision with non-categorical variables.

<B> NEED TO FIX OHEdict:  should only pull categories out of TRAINING set, NOT entire set.  Otherwise it's cheating.

In [52]:
OHEdict = {(0, v): k+3 for (k, v) in enumerate(srDigestR)}

print OHEdict

OHEdict[(0,'politics')]

Create OHETrainData.

In [55]:
def createLabeledPoint(point,numFeats):
    label = point[0]
    feats = point[1:]
    sv = SparseVector(numFeats, feats)
    # print sv
    return LabeledPoint(label, sv)

In [56]:
len(OHEdict)+3

35

createLabeledPoint((1, (0, 36), (1, -11.0), (2, 811), (25, 1)),   len(OHEdict+3  )

In [57]:
numFeats = len(OHEdict)+3
OHETrainData = (rawTrainData.map(lambda (label, t1, t2, t3, sr):
                                       (label, t1, t2, t3, (OHEdict[(0,sr)], 1) ))
                            .map(lambda point:  createLabeledPoint(point, numFeats))
                )

OHETrainData.take(10)

(Create OHEValData and OHETestData; skip for now)

# 8.0 Run logistic regression

Set up hyperparameters

In [59]:
# fixed hyperparameters
numIters = 50
stepSize = 10.
regParam = 1e-6
regType = 'l2'
includeIntercept = True

Run logistic regression

In [60]:
model0 = LogisticRegressionWithSGD.train(OHETrainData, iterations = numIters, step = stepSize,
                                        regParam = regParam, regType = regType,
                                        intercept = includeIntercept)
sortedWeights = sorted(model0.weights)
# print sortedWeights[:5], model0.intercept

[-3742.7305146647359, -0.11871780492023463, -0.10704296413708338, -0.037927471870862851, -0.021060388933347763] 9.8895531871
